In [45]:
import cv2
import keyboard
import numpy as np

def object_detection(video_path=None, object_detector=cv2.createBackgroundSubtractorMOG2(), contour_area_threshold=100):
    # Choose the video source
    if video_path:
        cap = cv2.VideoCapture(video_path)  # Use video file
    else:
        cap = cv2.VideoCapture(0)  # Use live video (camera)
    
    while True:
        # Capture each frame
        ret, frame = cap.read()
        parallelogram_left_points = np.array([[0, 0], [0, 90],  [80,45],[90, 0],], dtype=np.int32)
        parallelogram_center_points = np.array([[80,45],[90, 0], [200, 0],[150,45]], dtype=np.int32)
        parallelogram_right_points = np.array([[200, 0],[150,45],[255,100], [255, 0]], dtype=np.int32)

        frame = cv2.resize(frame,(224,224))
        # If frame reading failed, exit loop
        if not ret:
            print("Video finished or camera feed not available.")
            break
        cv2.fillPoly(frame, [parallelogram_left_points], (255, 255, 255))
        cv2.fillPoly(frame, [parallelogram_center_points], (255, 255, 255))
        cv2.fillPoly(frame, [parallelogram_right_points], (255, 255, 255))
        # Apply background subtraction to get the masks for MOG2 and KNN
        mask = object_detector.apply(frame)
        
        # Find contours of the detected objects for both MOG2 and KNN
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw bounding boxes for objects with a contour area larger than the threshold
        for contour in contours:
            if cv2.contourArea(contour) > contour_area_threshold:
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box for MOG2

        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # Stack the original frame and the mask horizontally
        stacked_frame = cv2.hconcat([frame, mask_bgr])
        
        # Display the stacked frames
        cv2.imshow('Feed', stacked_frame)

        # Check if 'ESC' key is pressed using the keyboard module
        cv2.waitKey(1)
        if keyboard.is_pressed('esc'):
            print("ESC pressed, exiting...")
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Example usage
object_detection(
    video_path='./test.mp4',           # Pass video path or None for live video
    contour_area_threshold=500  # Minimum contour area to filter small objects
)

ESC pressed, exiting...


In [ ]:
import cv2
import keyboard
import numpy as np
import math

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

# Function to calculate speed in km/h
def calculate_speed_in_kmh(point1, point2, scale_factor, fps):
    # Calculate pixel distance between two points
    pixel_distance = np.linalg.norm(np.array(point2) - np.array(point1))
    
    # Convert pixel distance to meters using scale factor (pixels to meters)
    distance_in_meters = pixel_distance * scale_factor
    
    # Time per frame (seconds)
    time_per_frame = 1 / fps
    
    # Calculate speed in meters per second (m/s)
    speed_mps = distance_in_meters / time_per_frame
    
    # Convert speed to km/h (1 m/s = 3.6 km/h)
    speed_kmh = speed_mps * 3.6
    
    return speed_kmh

def object_detection(video_path=None, object_detector=cv2.createBackgroundSubtractorMOG2(), 
                     contour_area_threshold=100, scale_factor=0.05):
    # Choose the video source
    if video_path:
        cap = cv2.VideoCapture(video_path)  # Use video file
    else:
        cap = cv2.VideoCapture(0)  # Use live video (camera)

    # Get frame rate of the video
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second

    # Initialize variables to store previous vehicle positions
    previous_positions = {}

    while True:
        # Capture each frame
        ret, frame = cap.read()
        parallelogram_left_points = np.array([[0, 0], [0, 90], [80, 45], [90, 0]], dtype=np.int32)
        parallelogram_center_points = np.array([[80, 45], [90, 0], [200, 0], [150, 45]], dtype=np.int32)
        parallelogram_right_points = np.array([[200, 0], [150, 45], [255, 100], [255, 0]], dtype=np.int32)

        frame = cv2.resize(frame, (224, 224))
        # If frame reading failed, exit loop
        if not ret:
            print("Video finished or camera feed not available.")
            break

        # Fill parallelogram regions (optional - can be removed if not required)
        cv2.fillPoly(frame, [parallelogram_left_points], (255, 255, 255))
        cv2.fillPoly(frame, [parallelogram_center_points], (255, 255, 255))
        cv2.fillPoly(frame, [parallelogram_right_points], (255, 255, 255))

        # Apply background subtraction to get the masks for MOG2
        mask = object_detector.apply(frame)

        # Find contours of the detected objects
        contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        # To store current frame's positions of detected vehicles
        current_positions = {}

        # Draw bounding boxes for objects with a contour area larger than the threshold
        for idx, contour in enumerate(contours):
            if cv2.contourArea(contour) > contour_area_threshold:
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box for detected objects

                # Calculate the center point of the bounding box (assumed as the vehicle position)
                center_point = (x + w // 2, y + h // 2)

                # Save current position for the vehicle (indexed by contour id)
                current_positions[idx] = center_point

                # Calculate speed if the vehicle was detected in the previous frame
                if idx in previous_positions:
                    # Get previous position
                    prev_center_point = previous_positions[idx]

                    # Calculate speed in km/h using scale factor (pixels to meters) and fps
                    speed_kmh = calculate_speed_in_kmh(prev_center_point, center_point, scale_factor, fps)

                    # Display the speed on the frame
                    cv2.putText(frame, f"Speed: {int(speed_kmh)} km/h", (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    print(f"Speed: {int(speed_kmh)} km/h")

        # Update the previous positions for the next frame
        previous_positions = current_positions.copy()

        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # Stack the original frame and the mask horizontally
        stacked_frame = cv2.hconcat([frame, mask_bgr])

        # Display the stacked frames
        cv2.imshow('Vehicle Detection and Speed', stacked_frame)

        # Check if 'ESC' key is pressed using the keyboard module
        cv2.waitKey(1)
        if keyboard.is_pressed('esc'):
            print("ESC pressed, exiting...")
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Example usage
object_detection(
    video_path='./test.mp4',           # Pass video path or None for live video
    contour_area_threshold=1000,        # Minimum contour area to filter small objects
    scale_factor=0.08                  # Scale factor (meters per pixel)
)


Speed: 1527 km/h
Speed: 69 km/h
Speed: 190 km/h
Speed: 0 km/h
Speed: 12 km/h
Speed: 968 km/h
Speed: 971 km/h
Speed: 0 km/h
Speed: 43 km/h
Speed: 79 km/h
Speed: 88 km/h
Speed: 101 km/h
Speed: 48 km/h
Speed: 48 km/h
Speed: 62 km/h
Speed: 57 km/h
Speed: 65 km/h
Speed: 12 km/h
Speed: 90 km/h
Speed: 77 km/h
Speed: 52 km/h
Speed: 27 km/h
Speed: 54 km/h
